# Chains

### LLMChain — the foundation

In [20]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

prompt = PromptTemplate(
    input_variables=["topic", "audience"],
    template=(
        "You are a helpful writing assistant.\n"
        "Write a catchy YouTube title about {topic} for {audience}. "
        "Keep it under 75 characters."
    )
)



** get prompt -> call llm **


In [21]:
prompt_text = prompt.invoke({"topic": "LangChain Chains", "audience": "Python developers"})
print("prompt : " ,prompt_text)
result = llm.invoke(prompt_text)
print("Result : ", result.content)

prompt :  text='You are a helpful writing assistant.\nWrite a catchy YouTube title about LangChain Chains for Python developers. Keep it under 75 characters.'
Result :  "Unlocking LangChain: Supercharge Your Python Development!"


In [22]:
title_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
title_chain.run(topic="LangChain Chains", audience="Python developers")



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful writing assistant.
Write a catchy YouTube title about LangChain Chains for Python developers. Keep it under 75 characters.

> Finished chain.


'"Unlocking LangChain: Essential Chains Every Python Developer Must Know!"'

### SimpleSequentialChain — pass string forward

In [23]:
from langchain.chains import SimpleSequentialChain

prompt1 = PromptTemplate.from_template(
    "Write a clicky YouTube title about: {topic}"
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

prompt2 = PromptTemplate.from_template(
    "Write a youtube video description for :\n{title}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

pipeline = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

outline = pipeline.run("LangChain Chains in Python")
print(outline)




> Entering new SimpleSequentialChain chain...
"Unlock the Power of LangChain: Build Smart Python Chains in Minutes!"
🌟 **Unlock the Power of LangChain: Build Smart Python Chains in Minutes!** 🌟

Welcome to our latest tutorial where we dive deep into the world of LangChain! 🚀 In this video, you'll learn how to harness the incredible capabilities of LangChain to create intelligent Python chains effortlessly. Whether you're a beginner or an experienced developer, this step-by-step guide will empower you to build smart applications in no time!

🔍 **What You’ll Learn:**
- What LangChain is and why it’s a game-changer for Python developers
- How to set up your environment for LangChain development
- Step-by-step instructions to create your first smart chain
- Tips and tricks for optimizing your chains for performance
- Real-world use cases and examples to inspire your projects

💡 **Who Is This For?**
- Python developers looking to enhance their skills
- Data scientists wanting to streamlin

### SequentialChain — named inputs & outputs

In [24]:
from langchain.chains import SequentialChain

title_prompt = PromptTemplate.from_template(
    "Generate a concise title about {topic} for {audience}."
)
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

desc_prompt = PromptTemplate.from_template(
    "Write a YouTube description (80–120 words) for the title:\n{title}\n"
    "Audience: {audience}\nInclude 3 hashtags."
)
desc_chain = LLMChain(llm=llm, prompt=desc_prompt, output_key="description")

kw_prompt = PromptTemplate.from_template(
    "Give 12 comma-separated SEO keywords for:\nTitle: {title}\nDesc: {description}"
)
kw_chain = LLMChain(llm=llm, prompt=kw_prompt, output_key="keywords")

seq = SequentialChain(
    chains=[title_chain, desc_chain, kw_chain],
    input_variables=["topic", "audience"],
    output_variables=["title", "description", "keywords"],
    verbose=True
)

result = seq({"topic": "LangChain Chains", "audience": "Python devs"})
print(result)




> Entering new SequentialChain chain...

> Finished chain.
{'topic': 'LangChain Chains', 'audience': 'Python devs', 'title': '"Mastering LangChain: A Python Developer\'s Guide to Building Intelligent Chains"', 'description': "Unlock the power of LangChain in your Python projects with our comprehensive guide! In this video, we’ll walk you through the essentials of building intelligent chains that enhance your applications. Whether you're a seasoned developer or just starting out, you'll learn how to leverage LangChain's capabilities to streamline workflows, integrate APIs, and create dynamic applications. Join us as we explore practical examples, best practices, and tips to elevate your coding skills. Don’t forget to like, subscribe, and hit the bell icon for more insights into Python development! \n\n#LangChain #PythonDevelopment #CodingTutorials", 'keywords': 'LangChain, Python development, intelligent chains, API integration, coding tutorials, Python projects, workflow automation, 

### TransformChain — inject custom Python logic

In [9]:
from langchain.chains import TransformChain
from typing import Dict

def to_slug(inputs: Dict) -> Dict:
    title = inputs["title"]
    slug = (
        title.lower()
        .replace("&", "and")
        .replace("—", "-")
        .replace(" ", "-")
    )
    return {"slug": slug}

slug_chain = TransformChain(
    input_variables=["title"],
    output_variables=["slug"],
    transform=to_slug
)

full = SequentialChain(
    chains=[title_chain, slug_chain, desc_chain, kw_chain],
    input_variables=["topic", "audience"],
    output_variables=["title", "slug", "description", "keywords"],
    verbose=True
)

print(full({"topic": "LangChain Chains", "audience": "Beginners"}))




> Entering new SequentialChain chain...

> Finished chain.
{'topic': 'LangChain Chains', 'audience': 'Beginners', 'title': '"Getting Started with LangChain Chains: A Beginner\'s Guide"', 'slug': '"getting-started-with-langchain-chains:-a-beginner\'s-guide"', 'description': "Welcome to our beginner-friendly guide on LangChain Chains! In this video, we'll walk you through the fundamentals of LangChain, a powerful framework for building applications with language models. Whether you're new to programming or just starting with AI, this tutorial will help you understand the basics of creating and managing chains effectively. We'll cover key concepts, practical examples, and tips to get you up and running in no time. Join us on this exciting journey into the world of LangChain! Don’t forget to like, subscribe, and hit the notification bell for more tutorials! \n\n#LangChain #AI #ProgrammingForBeginners", 'keywords': "LangChain, LangChain tutorial, beginner's guide to LangChain, AI programm

# LCEL (LangChain Expression Language)

### Think of LCEL as LEGO pipes for AI.

**Each block (prompt, model, tiny Python function, parser) has the same shape.**

You snap blocks together with a simple pipe |.
Then you run it the same way every time: invoke (one), batch (many), stream (piece by piece).

### The Idea (in one minute)

**Blocks:** Prompt → Model → (optional) Post-processor (parser)

**Pipe:** Use | to connect blocks into a mini-pipeline.

**Unified API:** No matter what you built, you call it with .invoke, .batch, or .stream.

In [10]:
chain = prompt | llm 

# one-off call (equivalent to .run)
result = chain.invoke({"topic": "LangChain Chains", "audience": "Python developers"})
print(result.content)

content='"Unlocking LangChain: Essential Chains Every Python Developer Needs!"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 35, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBlqfuWbrm8HAq0WQb0gkUj588Esg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--b73287c8-f9ec-41fe-9666-24d3e482a864-0' usage_metadata={'input_tokens': 35, 'output_tokens': 13, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Parallel chains 

Idea: fan out the same input to several blocks concurrently and get a dict back.


In [12]:
# pip install langchain langchain-openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Blocks
title_prompt = ChatPromptTemplate.from_template("Write a clicky YouTube title about: {topic}")
outline_prompt = ChatPromptTemplate.from_template("Create a 5-bullet outline for a video titled: {topic}")
hashtags_prompt = ChatPromptTemplate.from_template("Suggest 8 short hashtags for: {topic}")

title_chain = title_prompt | llm | StrOutputParser()
outline_chain = outline_prompt | llm | StrOutputParser()
hashtags_chain = hashtags_prompt | llm | StrOutputParser()

# Parallel fan-out; returns {"title": ..., "outline": ..., "hashtags": ...}
parallel = RunnableParallel(
    title=title_chain,
    outline=outline_chain,
    hashtags=hashtags_chain,
)

res = parallel.invoke({"topic": "LCEL basics"})
print(res["title"])
print(res["outline"])
print(res["hashtags"])


"Unlocking LCEL Basics: Your Ultimate Guide to Mastering Language Learning!"
### Video Title: LCEL Basics

#### Outline:

1. **Introduction to LCEL**
   - Definition of LCEL (Language and Content Integrated Learning)
   - Importance and relevance in modern education
   - Overview of the video content

2. **Key Principles of LCEL**
   - Integration of language learning with subject content
   - Focus on communicative competence and critical thinking
   - Emphasis on student-centered learning and engagement

3. **Benefits of LCEL**
   - Enhanced language proficiency through contextual learning
   - Development of cognitive skills and content knowledge
   - Preparation for real-world applications and multicultural understanding

4. **Implementation Strategies**
   - Effective teaching methods and approaches (e.g., project-based learning, collaborative tasks)
   - Tips for educators on designing LCEL activities
   - Resources and tools to support LCEL in the classroom

5. **Conclusion and 

# Conditional Chains

In [25]:
from langchain_core.runnables import RunnableBranch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

paraphrase_prompt = ChatPromptTemplate.from_template(
    "Paraphrase this in one snappy line:\n\n{text}"
)
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize this into exactly 5 bullets:\n\n{text}"
)

paraphrase_chain = paraphrase_prompt | llm | StrOutputParser()
summary_chain   = summary_prompt   | llm | StrOutputParser()

is_short = lambda inp: len(inp["text"].split()) < 60

router = RunnableBranch(
    (is_short, paraphrase_chain),# if True → paraphrase
    summary_chain           # else → summary?
)

print(router.invoke({"text": "LCEL is a simple pipe to connect prompts, models, and parsers."}))


LCEL seamlessly links prompts, models, and parsers.


In [26]:
# print(router.invoke({"text": "LCEL is a simple pipe to connect prompts, models, and parsers."}))
print(router.invoke({"text": "Condition chaining (e.g., with LCEL’s RunnableBranch) lets you route an input through exactly one path based on rules you define. You provide ordered predicate→chain pairs; at runtime the predicates are tested top-to-bottom, the first True wins, and only that subchain runs (no wasted tokens). A final fallback branch handles everything else (API detail: some versions pass it as the last positional arg, newer ones allow default=). Use it for things like short-vs-long handling, language detection, or “already structured?” checks. For reliability, keep predicates fast and deterministic, and make all branches return the same output type so downstream steps don’t break. Condition chains work with the same unified methods—invoke (one), batch (many), and stream (piecewise)—and you can tag/trace them like any other runnable."}))

- Condition chaining with LCEL’s RunnableBranch routes input through a single path based on defined rules using ordered predicate→chain pairs.
- At runtime, predicates are tested in order, with the first True result determining which subchain runs, minimizing wasted tokens.
- A fallback branch handles any unmatched conditions, with API variations allowing it to be passed as the last positional argument or as a default.
- Use cases include handling short vs. long inputs, language detection, and checks for pre-structured data.
- For reliability, ensure predicates are fast and deterministic, and maintain consistent output types across branches; condition chains integrate with invoke, batch, and stream methods.


- Condition chaining with LCEL’s RunnableBranch allows routing of input through a single defined path based on custom rules.
- Users provide ordered predicate→chain pairs, with the first True predicate determining which subchain runs, minimizing wasted tokens.
- A fallback branch is available for handling unmatched conditions, with API variations for its implementation.
- Ideal use cases include short-vs-long handling, language detection, and checks for pre-structured data.
- For reliability, ensure predicates are fast and deterministic, and maintain consistent output types across branches to avoid downstream issues.
